In [ ]:
# Workaround for training output not visible in JupyterNotebook https://github.com/microsoft/vscode-jupyter/issues/13163
from IPython.display import clear_output, DisplayHandle
def update_patch(self, obj):
    clear_output(wait=True)
    self.display(obj)
DisplayHandle.update = update_patch

## NLP wih RNNs. 

A language model is trained to guess the next word in a given text, based on text it has read before. This is called self supervised learning. 
The IMDB example used a language model trained on Wikipedia, but training on a corpus of target text (in this case IMDB) produces much better results. In this example the IMDB dataset will have a lot more different words, slang, and names that aren't in the Wikipedia dataset. 

The process goes: 

Tokenization - converting the text into tokens which are almost words, sometimes words and sometimes parts of words. Can be subwords or characters too. 
Numericalization - Make a list of all unique words that appear (the vocabulary) and convert each into a number. 
Language model data loader creation - create an independent variable which is the sequence of worsd from 1 to n-1, and a dependent variable which is from 2 to n. 
Lanauge model creation - Recurrent Neural Network to create an LM which takes large inputs. 




## Tokenization

In [ ]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

In [ ]:
txt = files[5].open().read(); 
txt[0:150]

In [ ]:
# Now use the WordTokenizer to tokenize words in the text. 
spacy = WordTokenizer()
tokens = first(spacy([txt]))
print(coll_repr(tokens, 30))

In [ ]:
tokenizer = Tokenizer(spacy)
print(coll_repr(tokenizer(txt), 31))

The `xx` is not common in English, so it's used as a prefix to indicate special tokens here.   The `xxbos` indicates start of a new text. (Beginning of Stream). `xxmaj` means the next word begins with a capital.  `xxunk` is an unknown word. 

Similarly there is `xxrep` to indicate repeated characters. 

In [ ]:
print(coll_repr(tokenizer("I like turtles!!!!"), 31))

The tokenization helps with model training, letting it recognize important parts of a sentence. 

In [ ]:
# See the rules
defaults.text_proc_rules

## Subword tokenization

Another approach to tokenization, instead of full words, is subwords. Subword is useful for languages like Chinese, Japanese because they don't necessarily use spaces or have the same definition of word as in English. Similarly, Turkish, Hungarian and German can add many subwords without spaces to make new words. 

Analyze a corpus of documents to find the most commonly occuring groups of letters (vocab)
Then, tokenize the corpus using this vocabulary of 'subword units'

In [ ]:
txts = L(o.open().read() for o in files[:2000])

In [ ]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])   


In [ ]:
subword(1000)